In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import utils.visualizer as uv
from tqdm.notebook import tqdm

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
plt.rcParams["animation.html"] = "jshtml"

## Константы

In [ ]:
# == общие ==
mu = 0.054
sigma = 5.67e-8
emissivity = 0.99

# == воздух ==
rho_a = 1.28
c_pa = 1.01e3
C_sh = 1.0e-3

# == вода ==
c_w = 3.99e3
rho_w = 1023.0
S_w = 30.0
Tf_w = -mu*S_w

# == лёд ==
c0_i = 2.06e3
rho_i = 917.0
# S_i = 5.0
# Tf_i = -mu*S_i
L0_i = 3.34e5
kappa_i = 1.5
albedo_i = 0.6
i0_i = 0.15

# == снег ==
c0_s = 2.06e3
rho_s = 330.0
k0_s = 0.31
L0_s = 3.33e5
kappa_s = 10
albedo_s = 0.8
i0_s = 0.08

## Функции

In [ ]:
# == общие ==
F_lw = lambda T_atm: sigma*(T_atm + 273.15)**4
F_su = lambda T, T_atm, albedo, i0, F_sw, u_a: (1 - albedo)*(1 - i0)*F_sw\
                                             + rho_a*c_pa*C_sh*u_a*(T_atm - T)\
                                             + emissivity*(F_lw(T_atm) - sigma*(T + 273.15)**4)
                                        
F_bt = lambda T, T_ocn: 0.0

# == лёд ==
Tf_i = lambda S_i: -mu*S_i
k_i = lambda T, S_i: rho_i/917.0 * (2.11 - 0.011*T + (0.09*S_i/T if T != 0 else 0))
c_i = lambda T, T_old, S_i: c0_i - L0_i*Tf_i(S_i)/(T*T_old)
E_i = lambda T, S_i: c0_i*(T - Tf_i(S_i)) - L0_i*(1.0 - Tf_i(S_i)/T) + c_w*Tf_i(S_i)
L_i = lambda T, S_i: c0_i*(T - Tf_i(S_i)) - L0_i*(1.0 - Tf_i(S_i)/T)

# == снег ==
k_s = lambda T, S_s=0: k0_s
c_s = lambda T, T_old, S_s=0: c0_s
E_s = lambda T, S_s=0: c0_s*T - L0_s
L_s = lambda T, S_s=0: c0_s*T - L0_s

In [ ]:
plt.figure(figsize=(15, 10))
plt.title("Different thermal conductivity parametrizations, S = 2", size=25)
T = np.linspace(-30.0, Tf_i(2), 100)
plt.plot(T, [k_i(t, 2) for t in T], label='bubbly brine', lw=3)
plt.plot(T, 2.03 + 0.1172*2/T, label='Untersteiner', lw=3)
plt.xlabel(r'Temperature, $^{\circ}C$', size=20)
plt.ylabel(r'Thermal Conductivity, $\dfrac{W}{m * K}$', size=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.legend(prop={'size':20})
plt.grid(ls='--')
plt.savefig('imgs/parametrizations.png')
plt.show()

## Код

In [ ]:
def secant_solver(f, x0, x1, tol=1e-9, max_it=1000):
    x_new = x1
    x_old = x0
    it = 0
    while abs(x_new - x_old) > tol and it < max_it:
        it += 1
        temp = x_new
        x_new = x_new - f(x_new)*(x_new-x_old)/(f(x_new) - f(x_old))
        x_old = temp
    
    if it == max_it:
        raise Exception("secant solver won't converge!")
        
    return x_new, it

In [ ]:
def thomas_solver(T_under, T_diag, T_over, rhs):
    assert len(T_under) == len(T_diag) - 1
    assert len(T_over) == len(T_diag) - 1
    assert len(T_diag) == len(rhs)
    
    sol = np.zeros(len(T_diag));
    c_new = np.zeros(len(T_diag) - 1);
    d_new = np.zeros(len(T_diag));

    # forward iteration
    c_new[0] = T_over[0]/T_diag[0];
    for i in range(1, len(T_diag) - 1):
        c_new[i] = T_over[i]/(T_diag[i] - T_under[i-1]*c_new[i-1])

    # backward iteration
    d_new[0] = rhs[0]/T_diag[0]
    for i in range(1, len(T_diag)):
        d_new[i] = (rhs[i] - T_under[i-1]*d_new[i-1])/(T_diag[i] - T_under[i-1]*c_new[i-1])

    # final solution
    sol[len(T_diag) - 1] = d_new[len(T_diag)-1]
    for i in range(len(T_diag) - 2, -1, -1): 
        sol[i] = d_new[i] - c_new[i]*sol[i+1]

    return sol

In [ ]:
def Update_dz(dz_cells_old, omega_ocn, omega_atm, time_step):
    dz_old = dz_cells_old.sum()
    return dz_cells_old - time_step*(omega_atm - omega_ocn)*dz_cells_old/dz_old

In [ ]:
def compute_radiation_nodes(dzi, F_sw, dzs=None):
    dzi_extended = np.concatenate(([0], dzi[::-1]))
    if dzs is None:
        radiation_nodes_ice = F_sw*(1-albedo_i)*i0_i*np.exp(-kappa_i*np.cumsum(dzi_extended))
        return radiation_nodes_ice[::-1]
    else:
        dzs_extended = np.concatenate(([0], dzs[::-1]))
        radiation_nodes_snow = F_sw*(1-albedo_s)*i0_s*np.exp(-kappa_s*np.cumsum(dzs_extended))
        radiation_nodes_ice = F_sw*(1-albedo_s)*i0_s*np.exp(-kappa_i*np.cumsum(dzi_extended)\
                                                            - sum(kappa_s*dzs))
        return radiation_nodes_snow[::-1], radiation_nodes_ice[::-1]

In [ ]:
def k_eff_node(k_low, k_high, dz_low, dz_high):
    return 2*k_low*k_high/(k_low*dz_high + k_high*dz_low)

In [ ]:
def get_matrix_upwind(T_cells_prev, T_cells_old,
                      T_ice_ocn_new, T_ice_ocn_prev, T_ice_ocn_old,
                      T_ice_atm_new, T_ice_atm_prev, T_ice_atm_old,
                      omega_ice_ocn, omega_ice_atm,
                      dz_cells_new, dz_cells_old,
                      salinity_cells,
                      radiation_nodes,
                      E, c, k, rho,
                      time_step, is_snow=False):
    
    N = len(T_cells_old)
    
    # проверка идентичности размеров массивов температур
    assert len(T_cells_prev) == N
    
    # проверка размерности массивов толщин ячеек
    assert len(dz_cells_new) == N
    assert len(dz_cells_old) == N
    
    # проверка размерности массива радиации
    assert len(radiation_nodes) == (N + 1)
    
    # расчет коэффициентов интерполяции (в каждом узле будет 2 коэффициента,
    # в первом и последнем узле коэффициенты нулевые) 
    a = np.zeros(N+1)
    b = np.zeros(N+1)
    
    for i in range(1, N):
        a[i] = dz_cells_new[i]/(dz_cells_new[i-1] + dz_cells_new[i])
        b[i] = dz_cells_new[i-1]/(dz_cells_new[i-1] + dz_cells_new[i])
    
    
    # расчет приближений коэффициента теплопроводности, теплоемкости и температуры в узлы
    k_nodes = np.zeros(N+1)
    
    if not is_snow:
    
        for i in range(1, N):
            k_nodes[i] = a[i]*k(T_cells_prev[i-1], salinity_cells[i-1]) + \
                         b[i]*k(T_cells_prev[i], salinity_cells[i])

        k_nodes[0] = k(T_cells_prev[0], salinity_cells[0])
        k_nodes[-1] = k(T_cells_prev[-1], salinity_cells[-1])

        c_ice_ocn = c(T_ice_ocn_prev, T_ice_ocn_old, salinity_cells[0])
        c_ice_atm = c(T_ice_atm_prev, T_ice_atm_old, salinity_cells[-1])

        E_ice_ocn = E(T_ice_ocn_old, salinity_cells[0])
        E_ice_atm = E(T_ice_atm_old, salinity_cells[-1])
        
    else:
        
        for i in range(1, N):
            k_nodes[i] = a[i]*k(T_cells_prev[i-1]) + \
                         b[i]*k(T_cells_prev[i])

        k_nodes[0] = k(T_cells_prev[0])
        k_nodes[-1] = k(T_cells_prev[-1])

        c_ice_ocn = c(T_ice_ocn_prev, T_ice_ocn_old)
        c_ice_atm = c(T_ice_atm_prev, T_ice_atm_old)

        E_ice_ocn = E(T_ice_ocn_old)
        E_ice_atm = E(T_ice_atm_old)
    
    # расчет узловых значений omega, теплоемкости и энтальпии
    omega_nodes = np.zeros(N+1)
    
    for i in range(0, N+1): 
        omega_nodes[i] = omega_ice_ocn\
                       + (sum(dz_cells_new[:i])/sum(dz_cells_new))*(omega_ice_atm - omega_ice_ocn)
    
    # расчет значений энтальпии, теплоемкости в ячейках
    E_cells = np.zeros(N)
    c_cells = np.zeros(N)
    
    for i in range(0, N):
        
        if not is_snow: 
            E_cells[i] = E(T_cells_old[i], salinity_cells[i])
            c_cells[i] = c(T_cells_prev[i], T_cells_old[i], salinity_cells[i])
        else:
            E_cells[i] = E(T_cells_old[i])
            c_cells[i] = c(T_cells_prev[i], T_cells_old[i])
        
    ### сборка диагоналей матрицы и вектора правой части
    A = np.zeros(N)
    B = np.zeros(N)
    C = np.zeros(N)
    RHS = np.zeros(N)
    
    # первая строка 
    B[0] = rho*c_cells[0]*dz_cells_new[0]/time_step + \
    (2.0*k_nodes[1])/(dz_cells_new[0] + dz_cells_new[1]) + (2.0*k_nodes[0])/(dz_cells_new[0])
    
    C[0] = -(2.0*k_nodes[1])/(dz_cells_new[0] + dz_cells_new[1])
    
    RHS[0] = rho*c_cells[0]*dz_cells_new[0]*T_cells_old[0]/time_step - \
    rho*E_cells[0]*(dz_cells_new[0] - dz_cells_old[0])/time_step + \
    (radiation_nodes[1] - radiation_nodes[0]) + \
    (2.0*k_nodes[0]*T_ice_ocn_new)/(dz_cells_new[0])
    
    if (omega_nodes[0] >= 0):
        RHS[0] += rho*c_ice_ocn*T_ice_ocn_new*omega_nodes[0] - \
        rho*(c_ice_ocn*T_ice_ocn_old - E_ice_ocn)*omega_nodes[0]
    else:
        B[0] += -rho*c_cells[0]*omega_nodes[0]
        RHS[0] += -rho*(c_cells[0]*T_cells_old[0] - E_cells[0])*omega_nodes[0]
            
    if (omega_nodes[1] >= 0):
        B[0] += rho*c_cells[0]*omega_nodes[1]
        RHS[0] += rho*(c_cells[0]*T_cells_old[0] - E_cells[0])*omega_nodes[1]
    else:
        C[0] += rho*c_cells[1]*omega_nodes[1]
        RHS[0] += rho*(c_cells[1]*T_cells_old[1] - E_cells[1])*omega_nodes[1]
        
    # серединные строки
    for i in range(1, N-1):
        
        A[i] = -(2.0*k_nodes[i])/(dz_cells_new[i-1] + dz_cells_new[i])
        
        B[i] = rho*c_cells[i]*dz_cells_new[i]/time_step + \
        (2.0*k_nodes[i+1])/(dz_cells_new[i] + dz_cells_new[i+1]) + \
        (2.0*k_nodes[i])/(dz_cells_new[i-1] + dz_cells_new[i]) 
        
        C[i] = -(2.0*k_nodes[i+1])/(dz_cells_new[i] + dz_cells_new[i+1])
        
        RHS[i] = rho*c_cells[i]*dz_cells_new[i]*T_cells_old[i]/time_step - \
        rho*E_cells[i]*(dz_cells_new[i] - dz_cells_old[i])/time_step + \
        (radiation_nodes[i+1] - radiation_nodes[i])
        
        if (omega_nodes[i] >= 0):
            A[i] += -rho*c_cells[i-1]*omega_nodes[i]
            RHS[i] += -rho*(c_cells[i-1]*T_cells_old[i-1] - E_cells[i-1])*omega_nodes[i]
        else:
            B[i] += -rho*c_cells[i]*omega_nodes[i]
            RHS[i] += -rho*(c_cells[i]*T_cells_old[i] - E_cells[i])*omega_nodes[i]
            
        if (omega_nodes[i+1] >= 0):
            B[i] += rho*c_cells[i]*omega_nodes[i+1]
            RHS[i] += rho*(c_cells[i]*T_cells_old[i] - E_cells[i])*omega_nodes[i+1]
        else:
            C[i] += rho*c_cells[i+1]*omega_nodes[i+1]
            RHS[i] += rho*(c_cells[i+1]*T_cells_old[i+1] - E_cells[i+1])*omega_nodes[i+1]
        
    # последняя строка
    A[N-1] = -(2.0*k_nodes[N-1])/(dz_cells_new[N-2] + dz_cells_new[N-1])

    B[N-1] = rho*c_cells[N-1]*dz_cells_new[N-1]/time_step + \
    (2.0*k_nodes[N])/(dz_cells_new[N-1]) + (2.0*k_nodes[N-1])/(dz_cells_new[N-2] + dz_cells_new[N-1])
    
    RHS[N-1] = rho*c_cells[N-1]*T_cells_old[N-1]*dz_cells_new[N-1]/time_step - \
    rho*E_cells[N-1]*(dz_cells_new[N-1] - dz_cells_old[N-1])/time_step - \
    (radiation_nodes[N] - radiation_nodes[N-1]) + \
    (2.0*k_nodes[N]*T_ice_atm_new)/(dz_cells_new[N-1])
    
    if (omega_nodes[N-1] >= 0):
        A[N-1] += -rho*c_cells[N-2]*omega_nodes[N-1]
        RHS[N-1] += -rho*(c_cells[N-2]*T_cells_old[N-2] - E_cells[N-2])*omega_nodes[N-1]
    else:
        B[N-1] += -rho*c_cells[N-1]*omega_nodes[N-1]
        RHS[N-1] += -rho*(c_cells[N-1]*T_cells_old[N-1] - E_cells[N-1])*omega_nodes[N-1]
            
    if (omega_nodes[N] >= 0):
        B[N-1] += rho*c_cells[N-1]*omega_nodes[N]
        RHS[N-1] += rho*(c_cells[N-1]*T_cells_old[N-1] - E_cells[N-1])*omega_nodes[N]
    else:
        RHS[N-1] += -rho*c_ice_atm*T_ice_atm_new*omega_nodes[N] + \
        rho*(c_ice_atm*T_ice_atm_old - E_ice_atm)*omega_nodes[N]
        
    return A[1:], B, C[:-1], RHS

In [ ]:
def get_matrix_upwind_2(T_cells_prev, T_cells_old,
                        T_ice_ocn_new, T_ice_ocn_prev, T_ice_ocn_old,
                        T_ice_atm_new, T_ice_atm_prev, T_ice_atm_old,
                        omega_ice_ocn, omega_ice_atm,
                        dz_cells_new, dz_cells_old,
                        salinity_cells,
                        radiation_nodes,
                        E, c, k, rho,
                        time_step, is_snow=False):
    
    N = len(T_cells_old)
    
    # проверка идентичности размеров массивов температур
    assert len(T_cells_prev) == N
    
    # проверка размерности массивов толщин ячеек
    assert len(dz_cells_new) == N
    assert len(dz_cells_old) == N
    
    # проверка размерности массива радиации
    assert len(radiation_nodes) == (N + 1)
    
    # расчет коэффициентов интерполяции (в каждом узле будет 2 коэффициента,
    # в первом и последнем узле коэффициенты нулевые) 
    a = np.zeros(N+1)
    b = np.zeros(N+1)
    
    if not is_snow:
    
        for i in range(1, N):
            k_nodes[i] = a[i]*k(T_cells_prev[i-1], salinity_cells[i-1]) + \
                         b[i]*k(T_cells_prev[i], salinity_cells[i])

        k_nodes[0] = k(T_cells_prev[0], salinity_cells[0])
        k_nodes[-1] = k(T_cells_prev[-1], salinity_cells[-1])

        c_ice_ocn = c(T_ice_ocn_prev, T_ice_ocn_old, salinity_cells[0])
        c_ice_atm = c(T_ice_atm_prev, T_ice_atm_old, salinity_cells[-1])

        E_ice_ocn = E(T_ice_ocn_old, salinity_cells[0])
        E_ice_atm = E(T_ice_atm_old, salinity_cells[-1])
        
    else:
        
        for i in range(1, N):
            k_nodes[i] = a[i]*k(T_cells_prev[i-1]) + \
                         b[i]*k(T_cells_prev[i])

        k_nodes[0] = k(T_cells_prev[0])
        k_nodes[-1] = k(T_cells_prev[-1])

        c_ice_ocn = c(T_ice_ocn_prev, T_ice_ocn_old)
        c_ice_atm = c(T_ice_atm_prev, T_ice_atm_old)

        E_ice_ocn = E(T_ice_ocn_old)
        E_ice_atm = E(T_ice_atm_old)
    
    # расчет узловых значений omega, теплоемкости и энтальпии
    omega_nodes = np.zeros(N+1)
    
    for i in range(0, N+1): 
        omega_nodes[i] = omega_ice_ocn\
                       + (sum(dz_cells_new[:i])/sum(dz_cells_new))*(omega_ice_atm - omega_ice_ocn)
    
    # расчет значений энтальпии, теплоемкости в ячейках
    E_cells = np.zeros(N)
    c_cells = np.zeros(N)
    
    for i in range(0, N):
        
        if not is_snow: 
            E_cells[i] = E(T_cells_old[i], salinity_cells[i])
            c_cells[i] = c(T_cells_prev[i], T_cells_old[i], salinity_cells[i])
        else:
            E_cells[i] = E(T_cells_old[i])
            c_cells[i] = c(T_cells_prev[i], T_cells_old[i])
        
    ### сборка диагоналей матрицы и вектора правой части
    A = np.zeros(N)
    B = np.zeros(N)
    C = np.zeros(N)
    RHS = np.zeros(N)
    
    # первая строка 
    B[0] = rho*c_cells[0]*dz_cells_new[0]/time_step + k_eff_nodes[0] + k_eff_nodes[1]
    
    C[0] = -k_eff_nodes[1]
    
    RHS[0] = rho*c_cells[0]*dz_cells_new[0]*T_cells_old[0]/time_step - \
    rho*E_cells[0]*(dz_cells_new[0] - dz_cells_old[0])/time_step + \
    (radiation_nodes[1] - radiation_nodes[0]) + \
    k_eff_nodes[0]*T_ice_ocn_new
    
    if (omega_nodes[0] >= 0):
        RHS[0] += +rho*c_ice_ocn*T_ice_ocn_new*omega_nodes[0] - \
        rho*(c_ice_ocn*T_ice_ocn_old - E_ice_ocn)*omega_nodes[0]
    else:
        B[0] += -rho*c_cells[0]*omega_nodes[0]
        RHS[0] += -rho*(c_cells[0]*T_cells_old[0] - E_cells[0])*omega_nodes[0]
            
    if (omega_nodes[1] >= 0):
        B[0] += rho*c_cells[0]*omega_nodes[1]
        RHS[0] += rho*(c_cells[0]*T_cells_old[0] - E_cells[0])*omega_nodes[1]
    else:
        C[0] += rho*c_cells[1]*omega_nodes[1]
        RHS[0] += rho*(c_cells[1]*T_cells_old[1] - E_cells[1])*omega_nodes[1]
        
    # серединные строки
    for i in range(1, N-1):
        
        A[i] = -k_eff_nodes[i]
        
        B[i] = rho*c_cells[i]*dz_cells_new[i]/time_step + k_eff_nodes[i] + k_eff_nodes[i+1]
        
        C[i] = -k_eff_nodes[i+1]
        
        RHS[i] = rho*c_cells[i]*dz_cells_new[i]*T_cells_old[i]/time_step - \
        rho*E_cells[i]*(dz_cells_new[i] - dz_cells_old[i])/time_step + \
        (radiation_nodes[i+1] - radiation_nodes[i])
        
        if (omega_nodes[i] >= 0):
            A[i] += -rho*c_cells[i-1]*omega_nodes[i]
            RHS[i] += -rho*(c_cells[i-1]*T_cells_old[i-1] - E_cells[i-1])*omega_nodes[i]
        else:
            B[i] += -rho*c_cells[i]*omega_nodes[i]
            RHS[i] += -rho*(c_cells[i]*T_cells_old[i] - E_cells[i])*omega_nodes[i]
            
        if (omega_nodes[i+1] >= 0):
            B[i] += rho*c_cells[i]*omega_nodes[i+1]
            RHS[i] += rho*(c_cells[i]*T_cells_old[i] - E_cells[i])*omega_nodes[i+1]
        else:
            C[i] += rho*c_cells[i+1]*omega_nodes[i+1]
            RHS[i] += rho*(c_cells[i+1]*T_cells_old[i+1] - E_cells[i+1])*omega_nodes[i+1]
        
    # последняя строка
    A[N-1] = -k_eff_nodes[N-1]

    B[N-1] = rho*c_cells[N-1]*dz_cells_new[N-1]/time_step + k_eff_nodes[N-1] + k_eff_nodes[N]
    
    RHS[N-1] = rho*c_cells[N-1]*T_cells_old[N-1]*dz_cells_new[N-1]/time_step - \
    rho*E_cells[N-1]*(dz_cells_new[N-1] - dz_cells_old[N-1])/time_step - \
    (radiation_nodes[N] - radiation_nodes[N-1]) + \
    k_eff_nodes[N]*T_ice_atm_new
    
    if (omega_nodes[N-1] >= 0):
        A[N-1] += -rho*c_cells[N-2]*omega_nodes[N-1]
        RHS[N-1] += -rho*(c_cells[N-2]*T_cells_old[N-2] - E_cells[N-2])*omega_nodes[N-1]
    else:
        B[N-1] += -rho*c_cells[N-1]*omega_nodes[N-1]
        RHS[N-1] += -rho*(c_cells[N-1]*T_cells_old[N-1] - E_cells[N-1])*omega_nodes[N-1]
            
    if (omega_nodes[N] >= 0):
        B[N-1] += rho*c_cells[N-1]*omega_nodes[N]
        RHS[N-1] += rho*(c_cells[N-1]*T_cells_old[N-1] - E_cells[N-1])*omega_nodes[N]
    else:
        RHS[N-1] += -rho*c_ice_atm*T_ice_atm_new*omega_nodes[N] + \
        rho*(c_ice_atm*T_ice_atm_old - E_ice_atm)*omega_nodes[N]
        
    return A[1:], B, C[:-1], RHS

In [ ]:
def concat_matrices(A_i, B_i, C_i, RHS_i,
                    A_s, B_s, C_s, RHS_s,
                    dzi, dzs,
                    k_i, k_s):
    
    # приближение второго порядка: T_i'z = alpha*T_is + beta*T_i-1/2 + gamma*T_i-3/2
    
    h1_i = dzi[-1]/2.0
    h2_i = dzi[-2]/2.0 + dzi[-1]
    
    h1_s = dzs[0]/2.0
    h2_s = dzi[1]/2.0 + dzi[0]
    
    alpha_i = (h1_i + h2_i)/(h1_i*h2_i)
    beta_i = -h2_i/(h1_i*(h2_i - h1_i))
    gamma_i = h1_i/(h2_i*(h2_i - h1_i))
    
    alpha_s = -(h1_s + h2_s)/(h1_s*h2_s)
    beta_s = h2_s/(h1_s*(h2_s - h1_s))
    gamma_s = -h1_s/(h2_s*(h2_s - h1_s))
    
    A_is = k_i*gamma_i
    B_is = k_i*beta_i
    C_is = k_i*alpha_i - k_s*alpha_s
    D_is = -k_s*beta_s
    E_is = -k_s*gamma_s
    RHS_is = 0
    
    # производим вычитание из промежуточной строки так, чтобы крайние элементы занулились
    
    # 1
    B_is -= A_is*B_i[-1]/A_i[-1]
    RHS_is -= A_is*RHS_i[-1]/A_i[-1]
    A_is = 0
    
    # 2
    D_is -= E_is*B_s[0]/C_s[0]
    RHS_is -= E_is*RHS_s[0]/C_s[0]
    E_is = 0
    
    # получаем трёхдиагональную матрицу, производим конкатенацию
    
    A_full = np.concatenate((A_i, [B_is, 0], A_s))
    B_full = np.concatenate((B_i, [C_is], B_s))
    C_full = np.concatenate((C_i, [0, D_is], C_s))
    RHS_full = np.concatenate((RHS_i, [RHS_is], RHS_s))
    
    return A_full, B_full, C_full, RHS_full

In [ ]:
def W_from_BC(T_node, T_cells,
              dz_cells,
              salinity_cells,
              k, F, 
              L,
              is_surface=False, rho=rho_i, time_step=None):
    
    if is_surface:
    
        h1 = dz_cells[-1]/2.0
        h2 = dz_cells[-1] + dz_cells[-2]/2.0
        # возможно, тут стоит учесть солёность
        grad = lambda T: (T*(h2**2 - h1**2) - T_cells[-1]*h2**2 + T_cells[-2]*h1**2)/(h1*h2*(h2-h1)) 
        residue = (k*grad(T_node) - F)*time_step
        omega = 0
        melted_layers = 0

        for T, dz, S in zip(T_cells[::-1], dz_cells[::-1], salinity_cells[::-1]):
            # print('melted:', melted_layers, 'residue:', residue, 'layer enthalpy:', rho*L(T)*dz)
            if residue <= rho*L(T, S)*dz:
                residue -= rho*L(T, S)*dz
                omega += dz/time_step
                melted_layers += 1
            else:
                break

        omega += residue/(rho*L(T, S)*time_step)
        return omega
        
    else:
        
        h1 = dz_cells[0]/2.0
        h2 = dz_cells[0] + dz_cells[1]/2.0
        grad = lambda T: (-T*(h2**2 - h1**2) + T_cells[0]*h2**2 - T_cells[1]*h1**2)/(h1*h2*(h2-h1))
        omega = (k*grad(T_node) - F)/(rho_i*L(T_cells[0], salinity_cells[0]))
        return omega

In [ ]:
def T_from_BC(T_cells, dz_cells, salinity_cells,
              k, omega,
              F, L,
              is_surface=False, rho=rho_i):
    
    # аппроксимация градиента на границе
    if is_surface:
        h1 = dz_cells[-1]/2.0
        h2 = dz_cells[-1] + dz_cells[-2]/2.0
        # возможно, тут стоит учесть солёность
        grad_su = lambda T: (T*(h2**2 - h1**2) - T_cells[-1]*h2**2 + T_cells[-2]*h1**2)/(h1*h2*(h2-h1))
        nonlin_func = lambda T: rho*L(T, salinity_cells[-1])*omega + F(T) - k*(grad_su(T))
        return secant_solver(nonlin_func, T_cells[-1]-10.0, Tf_i(salinity_cells[-1])+10.0)[0]
    
    else:
        h1 = dz_cells[0]/2.0
        h2 = dz_cells[0] + dz_cells[1]/2.0
        grad_su = lambda T: (-T*(h2**2 - h1**2) + T_cells[0]*h2**2 - T_cells[1]*h1**2)/(h1*h2*(h2-h1))
        nonlin_func = lambda T: rho_i*L(T, salinity_cells[0])*omega + F(T) - k*(grad_su(T))
        return secant_solver(nonlin_func, T_cells[0]-10.0, Tf_i(salinity_cells[0])+10.0)[0] 

In [ ]:
def ice_freezing(Ti, Tia, To, Ta, F_sw,
                 dzi, salinity,
                 N_pseudoiter,
                 time_step,
                 u_a,
                 tol=1e-6,
                 err_func=lambda T_old, T_prev, T_new: \
                 np.linalg.norm(T_new - T_prev)/np.linalg.norm(T_old)
                ):
    
    Ti_old = Ti
    Ti_new = Ti
    Tia_old = Tia
    Tia_new = Tia
    
    dzi_old = dzi
    dzi_new = dzi
    
    salinity_cells = salinity
    
    Tia_values = [Tia_old]
    current_err = np.inf
    surface_err = np.inf
    prev_surface_err = surface_err
    
    for pseudoiter in range(N_pseudoiter):
        
        Ti_prev = Ti_new
        Tia_prev = Tia_new
        
        prev_surface_err = surface_err
        
        omega_io = W_from_BC(Tf_w, Ti_prev,
                             dzi_new,
                             salinity_cells,
                             k_i(Tf_w, salinity_cells[0]),
                             F_bt(Tf_w, To), 
                             L_i,
                             is_surface=False
                            )

        # оценка Tia
        Tia_new = T_from_BC(Ti_prev, dzi_new,
                            salinity_cells,
                            k_i(Tia_prev, salinity_cells[-1]), 0.0,
                            lambda T: F_su(T, Ta, albedo_i, i0_i, F_sw, u_a), L_i,
                            is_surface=True
                           )
        
        # пересчет толщин слоев
        dzi_new = Update_dz(dzi_old, omega_io, 0.0, time_step)
        
        # пересчёт радиации
        radiation_nodes_ice = compute_radiation_nodes(dzi_new, F_sw)

        # перессчет температур в ячейках
        A, B, C, RHS = get_matrix_upwind(Ti_prev, Ti_old,
                                         Tf_w, Tf_w, Tf_w,
                                         Tia_new, Tia_prev, Tia_old,
                                         omega_io, 0.0,
                                         dzi_new, dzi_old,
                                         salinity_cells,
                                         radiation_nodes_ice,
                                         E_i, c_i, k_i, rho_i,
                                         time_step)
        
        Ti_new = thomas_solver(A, B, C, RHS)

        # оценка ошибки
        prev_surface_err = surface_err
        surface_err = abs(Tia_new - Tia_prev)/abs(Tia_old)
        
        if (surface_err < prev_surface_err):
            Tia_values.append(Tia_new)
        else:
            Tia_new = sum(Tia_values)/len(Tia_values)
        
        T_full_old = np.append(Ti_old, Tia_old)
        T_full_prev = np.append(Ti_prev, Tia_prev)
        T_full_new = np.append(Ti_new, Tia_new)
        err = err_func(T_full_old, T_full_prev, T_full_new)
        curr_err = err
        
        if current_err < tol:
            break
    
    return Ti_new, Tia_new, dzi_new

In [ ]:
def ice_melting(Ti, Tia_old, To, Ta, F_sw,
                dzi, salinity,
                N_pseudoiter,
                time_step,
                u_a,
                tol=1e-6,
                err_func=lambda T_old, T_prev, T_new: \
                np.linalg.norm(T_new - T_prev)/np.linalg.norm(T_old)):
    
    # инициализация текущих T
    Ti_old = Ti
    Ti_new = Ti
    
    dzi_old = dzi
    dzi_new = dzi
    
    salinity_cells = salinity
    
    current_err = np.inf
    
    for pseudoiter in range(N_pseudoiter):
    
        # инициализация текущих T
        Ti_prev = Ti_new

        # оценка omega на границе лед-океан
        omega_io = W_from_BC(Tf_w, Ti_prev,
                             dzi_new,
                             salinity_cells,
                             k_i(Tf_w, salinity_cells[0]),
                             F_bt(Tf_w, To), 
                             L_i,
                             is_surface=False)
        
        # оценка omega на границе лед-атмосфера
        omega_ia = W_from_BC(Tf_i(salinity_cells[-1]), Ti_prev,
                             dzi_new,
                             salinity_cells,
                             k_i(Tf_i(salinity_cells[-1]), salinity_cells[-1]),
                             F_su(Tf_i(salinity_cells[-1]), Ta, albedo_i, i0_i, F_sw, u_a),
                             L_i,
                             is_surface=True,
                             time_step=time_step)
        
        # пересчет толщин слоев
        dzi_new = Update_dz(dzi_old, omega_io, omega_ia, time_step)
        
        # пересчёт радиации
        radiation_nodes_ice = compute_radiation_nodes(dzi_new, F_sw)

        # перессчет температур в ячейках
        A, B, C, RHS = get_matrix_upwind(Ti_prev, Ti_old,
                                         Tf_w, Tf_w, Tf_w,
                                         Tf_i(salinity_cells[-1]), Tf_i(salinity_cells[-1]), Tia_old,
                                         omega_io, omega_ia,
                                         dzi_new, dzi_old,
                                         salinity_cells,
                                         radiation_nodes_ice,
                                         E_i, c_i, k_i, rho_i,
                                         time_step)
        
        Ti_new = thomas_solver(A, B, C, RHS)

        # оценка ошибки
        err = err_func(Ti_old, Ti_prev, Ti_new)
        curr_err = err
        
        if current_err < tol:
            break
            
    return Ti_new, dzi_new

In [ ]:
def snow_ice_freezing(Ti, Ts, Tis, Tsa, To, Ta, F_sw, 
                      dzi, dzs, salinity,
                      N_pseudoiter,
                      time_step,
                      p, u_a,
                      tol=1e-6,
                      err_func=lambda T_old, T_prev, T_new: \
                      np.linalg.norm(T_new - T_prev)/np.linalg.norm(T_old)
                     ):
    
    Ti_old = Ti
    Ti_new = Ti
    Ts_old = Ts
    Ts_new = Ts
    Tis_old = Tis
    Tis_new = Tis
    Tsa_old = Tsa
    Tsa_new = Tsa
    
    dzi_old = dzi
    dzi_new = dzi
    dzs_old = dzs
    dzs_new = dzs
    
    salinity_cells = salinity
    
    Tsa_values = [Tsa_old]
    current_err = np.inf
    surface_err = np.inf
    prev_surface_err = surface_err
    
    for pseudoiter in range(N_pseudoiter):
        
        Ti_prev = Ti_new
        Ts_prev = Ts_new
        Tis_prev = Tis_new
        Tsa_prev = Tsa_new
        
        prev_surface_err = surface_err
        
        omega_io = W_from_BC(Tf_w, Ti_prev,
                             dzi_new,
                             salinity_cells,
                             k_i(Tf_w, salinity_cells[0]),
                             F_bt(Tf_w, To), 
                             L_i,
                             is_surface=False
                            )

        # оценка Tsa
        Tsa_new = T_from_BC(Ts_prev, dzs_new,
                            salinity_cells,
                            k_s(Tsa_prev, salinity_cells[-1]), 0.0,
                            lambda T: F_su(T, Ta, albedo_s, i0_s, F_sw, u_a), L_s,
                            is_surface=True,
                            rho=rho_s
                           )
        
        omega_sa = 0
        
        # пересчет толщин слоев
        dzi_new = Update_dz(dzi_old, omega_io, 0.0, time_step)
        if Ta < 0:
            omega_sa -= p*rho_w/rho_s
            dzs_new = Update_dz(dzs_old, 0.0, omega_sa, time_step)
            
        # пересчёт радиации
        radiation_nodes_snow, radiation_nodes_ice = compute_radiation_nodes(dzi_new, F_sw, dzs_new)

        # перессчет температур в ячейках
        A_i, B_i, C_i, RHS_i = get_matrix_upwind(Ti_prev, Ti_old,
                                                 Tf_w, Tf_w, Tf_w,
                                                 Tis_new, Tis_prev, Tis_old,
                                                 omega_io, 0.0,
                                                 dzi_new, dzi_old,
                                                 salinity_cells,
                                                 radiation_nodes_ice,
                                                 E_i, c_i, k_i, rho_i,
                                                 time_step)
        
        A_s, B_s, C_s, RHS_s = get_matrix_upwind(Ts_prev, Ts_old,
                                                 Tis_new, Tis_prev, Tis_old,
                                                 Tsa_new, Tsa_prev, Tsa_old,
                                                 0.0, omega_sa,
                                                 dzs_new, dzs_old,
                                                 salinity_cells,
                                                 radiation_nodes_snow,
                                                 E_s, c_s, k_s, rho_s,
                                                 time_step, is_snow=True)
        
        A, B, C, RHS = concat_matrices(A_i, B_i, C_i, RHS_i,
                                       A_s, B_s, C_s, RHS_s,
                                       dzi_new, dzs_new,
                                       k_i(Tis_new, salinity_cells[-1]), k_s(Tis_new))
        
        T_vec_new = thomas_solver(A, B, C, RHS)
        
        Ti_new, Tis_new, Ts_new = T_vec_new[:len(Ti_old)],\
                                  T_vec_new[len(Ti_old)],\
                                  T_vec_new[len(Ti_old)+1:]

        # оценка ошибки
        prev_surface_err = surface_err
        surface_err = abs(Tsa_new - Tsa_prev)/(abs(Tsa_old) if Tsa_old != 0 else 1e-6)
        
        if (surface_err < prev_surface_err):
            Tsa_values.append(Tsa_new)
        else:
            Tsa_new = sum(Tsa_values)/len(Tsa_values)
        
        T_full_old = np.concatenate((Ti_old, [Tis_old], Ts_old, [Tsa_old]))
        T_full_prev = np.concatenate((Ti_prev, [Tis_prev], Ts_prev, [Tsa_prev]))
        T_full_new = np.concatenate((Ti_new, [Tis_new], Ts_new, [Tsa_new]))
        err = err_func(T_full_old, T_full_prev, T_full_new)
        curr_err = err
        
        if current_err < tol:
            break
    
    return Ti_new, Ts_new, Tis_new, Tsa_new, dzi_new, dzs_new

In [ ]:
def snow_melting(Ti, Ts, Tis, Tsa_old, To, Ta, F_sw,
                 dzi, dzs, salinity,
                 N_pseudoiter,
                 time_step,
                 p, u_a,
                 tol=1e-6,
                 err_func=lambda T_old, T_prev, T_new: \
                 np.linalg.norm(T_new - T_prev)/np.linalg.norm(T_old)):
    
    # инициализация текущих T
    Ti_old = Ti
    Ti_new = Ti
    Ts_old = Ts
    Ts_new = Ts
    Tis_old = Tis
    Tis_new = Tis
    
    dzi_old = dzi
    dzi_new = dzi
    dzs_old = dzs
    dzs_new = dzs
    
    salinity_cells = salinity
    
    T_vec_old = np.concatenate((Ti_old, [Tis_old], Ts_old))
    T_vec_new = np.concatenate((Ti_new, [Tis_new], Ts_new))
    current_err = np.inf
    
    for pseudoiter in range(N_pseudoiter):
    
        # инициализация текущих T
        Ti_prev = Ti_new
        Ts_prev = Ts_new
        Tis_prev = Tis_new
        T_vec_prev = T_vec_new

        # оценка omega на границе лед-океан
        omega_io = W_from_BC(Tf_w, Ti_prev,
                             dzi_new,
                             salinity_cells,
                             k_i(Tf_w, salinity_cells[0]),
                             F_bt(Tf_w, To), 
                             L_i,
                             is_surface=False)
        
        # оценка omega на границе лед-атмосфера
        omega_sa = W_from_BC(0.0, Ts_prev,
                             dzs_new,
                             salinity_cells,
                             k_s(0.0, salinity_cells[-1]),
                             F_su(0.0, Ta, albedo_s, i0_s, F_sw, u_a),
                             L_s,
                             is_surface=True,
                             rho=rho_s,
                             time_step=time_step)
        
        if Ta < 0:
            omega_sa -= p*rho_w/rho_s
        
        # пересчет толщин слоев
        dzi_new = Update_dz(dzi_old, omega_io, 0.0, time_step)
        dzs_new = Update_dz(dzs_old, 0.0, omega_sa, time_step)
        
        # пересчёт радиации
        radiation_nodes_snow, radiation_nodes_ice = compute_radiation_nodes(dzi_new, F_sw, dzs_new)

        # перессчет температур в ячейках
        A_i, B_i, C_i, RHS_i = get_matrix_upwind(Ti_prev, Ti_old,
                                                 Tf_w, Tf_w, Tf_w,
                                                 Tis_new, Tis_prev, Tis_old,
                                                 omega_io, 0.0,
                                                 dzi_new, dzi_old,
                                                 salinity_cells,
                                                 radiation_nodes_ice,
                                                 E_i, c_i, k_i, rho_i,
                                                 time_step)
        
        A_s, B_s, C_s, RHS_s = get_matrix_upwind(Ts_prev, Ts_old,
                                                 Tis_new, Tis_prev, Tis_old,
                                                 0.0, 0.0, Tsa_old,
                                                 0.0, omega_sa,
                                                 dzs_new, dzs_old,
                                                 salinity_cells,
                                                 radiation_nodes_snow,
                                                 E_s, c_s, k_s, rho_s,
                                                 time_step, is_snow=True)
        
        A, B, C, RHS = concat_matrices(A_i, B_i, C_i, RHS_i,
                                       A_s, B_s, C_s, RHS_s,
                                       dzi_new, dzs_new,
                                       k_i(Tis_new, salinity_cells[-1]), k_s(Tis_new))
        
        T_vec_new = thomas_solver(A, B, C, RHS)
        
        Ti_new, Tis_new, Ts_new = T_vec_new[:len(Ti_old)],\
                                  T_vec_new[len(Ti_old)],\
                                  T_vec_new[len(Ti_old)+1:]

        # оценка ошибки
        err = err_func(T_vec_old, T_vec_prev, T_vec_new)
        curr_err = err
        
        if current_err < tol:
            break
            
    return Ti_new, Ts_new, Tis_new, dzi_new, dzs_new

In [ ]:
def main_process(time_step, time_end,
                 N_pseudoiter,
                 Ti_init, Ts_init, Toi_init, Tis_init, Tsa_init,
                 dzi_init, dzs_init,
                 salinity,
                 snow_thickness_threshold
                ):
    
    time = 0.0
    
    Ti_old = Ti_init
    Ti_new = Ti_init
    Ts_old = Ts_init
    Ts_new = Ts_init
    Tis_old = Tis_init
    Tis_new = Tis_init
    Tsa_old = Tsa_init
    Tsa_new = Tsa_init
    
    
    dzi_old = dzi_init
    dzi_new = dzi_init
    dzs_old = dzs_init
    dzs_new = dzs_init
    
    salinity_cells = salinity
    
    dzi_arr = [dzi_init]
    dzs_arr = [dzs_init]
    timeline = [time]
    temp_oi_arr = [Tf_w]
    temp_ice_arr = [Ti_init]
    temp_is_arr = [Tis_new]
    temp_snow_arr = [Ts_init]
    temp_sa_arr = [Tsa_new]
    rho_ice_arr = [[rho_i]*len(dzi_init)]
    snow_filter = [sum(dzs_init) >= snow_thickness_threshold]
    
    while time < time_end:
        
        if time + time_step < time_end:
            time += time_step
        else:
            time_step = time_end - time
            time = time_end
            
        Ti_old, Ts_old, Tis_old, Tsa_old, dzi_old, dzs_old\
        = Ti_new, Ts_new, Tis_new, Tsa_new, dzi_new, dzs_new
        
        if sum(dzs_new) < snow_thickness_threshold:
            print('Time {:.1f} h.: Ice freezing...'.format(time/3600))
            Ti_new, Tis_new, dzi_new = ice_freezing(Ti_old, Tis_old, To(time), Ta(time), F_sw(time),
                                                    dzi_old,
                                                    salinity_cells,
                                                    N_pseudoiter,
                                                    time_step,
                                                    u_a(time)
                                                   )
            if Tis_new >= Tf_i(salinity_cells[-1]):
                print('Time {:.1f} h.: Ice melting...'.format(time/3600))
                Tis_new = Tf_i(salinity_cells[-1])
                Ti_new, dzi_new = ice_melting(Ti_old, Tis_old, To(time), Ta(time), F_sw(time),
                                              dzi_old,
                                              salinity_cells,
                                              N_pseudoiter,
                                              time_step,
                                              u_a(time)
                                             )
                
            # присыпем чуть снега сверху, если он падает
            if Ta(time) < 0:
                if sum(dzs_old) != 0:
                    dzs_new = Update_dz(dzs_old, 0.0, -p(time)*rho_w/rho_s, time_step)
                else:
                    dzs_new = np.full(len(dzs_old), time_step*p(time)*rho_w/rho_s/Ns)
                    
                if sum(dzs_new) > snow_thickness_threshold:
                    # инициализируем температуру появившегося снега
                    Tsa_new = Ta(time)
                    Ts_new = Tis_new + np.arange(0.5, Ns)/Ns * (Tsa_new - Tis_new)
                    
            snow_filter.append(False)
            
        else:
            print('Time {:.1f} h.: Snow-ice freezing...'.format(time/3600))
            Ti_new, Ts_new, Tis_new, Tsa_new, dzi_new, dzs_new =\
            snow_ice_freezing(Ti_old, Ts_old, Tis_old, Tsa_old, To(time), Ta(time), F_sw(time),
                              dzi_old, dzs_old,
                              salinity_cells,
                              N_pseudoiter,
                              time_step,
                              p(time), u_a(time)
                             )
            
            if Tsa_new >= 0:
                print('Time {:.1f} h.: Snow-ice melting...'.format(time/3600))
                Tsa_new = 0
                Ti_new, Ts_new, Tis_new, dzi_new, dzs_new =\
                snow_melting(Ti_old, Ts_old, Tis_old, Tsa_old, To(time), Ta(time), F_sw(time),
                             dzi_old, dzs_old,
                             salinity_cells,
                             N_pseudoiter,
                             time_step,
                             p(time), u_a(time)
                            )
                
            snow_filter.append(True)

        dzi_arr.append(dzi_new.copy())
        dzs_arr.append(dzs_new.copy())
        timeline.append(time)
        temp_oi_arr.append(Tf_w)
        temp_ice_arr.append(Ti_new.copy())
        temp_is_arr.append(Tis_new)
        temp_snow_arr.append(Ts_new.copy())
        temp_sa_arr.append(Tsa_new)
        rho_ice_arr.append([rho_i]*len(dzi_init))
    
    return dzi_arr, dzs_arr, timeline, temp_oi_arr, temp_ice_arr, temp_is_arr, temp_snow_arr, temp_sa_arr,\
           rho_ice_arr, snow_filter

## Эээээээээксперименты

### 0. Линейный профиль с закрепленными концами

In [ ]:
To = lambda t: Tf_w
Ta = lambda t: -30.0
p = lambda t: 0.0
F_sw = lambda t: 100.0
u_a = lambda t: 15.0
 
Tis_init = -20.0
ice_thickness_init = 2.0
Ti_func_init = lambda z: (z*Tis_init + (ice_thickness_init-z)*To(0.0))/ice_thickness_init - \
                          5*np.sin(2*np.pi/ice_thickness_init*z)

Ni = 20
dzi_init = np.arange(Ni, 0, -1)/sum(np.arange(Ni, 0, -1))*ice_thickness_init
ice_cells_init = dzi_init.cumsum() - dzi_init/2
salinity_init = np.full(Ni, 3.0)

In [ ]:
dzi_arr, dzs_arr,\
timeline,\
temp_oi_arr, temp_ice_arr, temp_is_arr, temp_snow_arr, temp_sa_arr,\
rho_ice_arr,\
snow_filter = main_process\
(
    time_step=3600,
    time_end=3600*100,
    N_pseudoiter=25,
    Ti_init=Ti_func_init(ice_cells_init),
    Ts_init=[],
    Toi_init=To(0.0),
    Tis_init=Tis_init,
    Tsa_init=0.0,
    dzi_init=dzi_init,
    dzs_init=[],
    salinity=salinity_init,
    snow_thickness_threshold=0.05               
)

In [ ]:
vis_0 = uv.animate(dzi_arr, dzs_arr,
                   temp_oi_arr, temp_ice_arr, temp_is_arr, temp_snow_arr, temp_sa_arr,
                   timeline,
                   rho_ice_arr, rho_w, rho_s,
                   snow_filter)

In [ ]:
vis_0.save('imgs/zero_anim.mp4', dpi=150)

### 1. Намерзание льда при постоянной погоде без осадков

In [ ]:
To = lambda t: Tf_w
Ta = lambda t: -30.0
p = lambda t: 0.0
F_sw = lambda t: 100.0
u_a = lambda t: 15.0
 
Tis_init = -15.0
Tsa_init = -20.0
ice_thickness_init = 2.0
snow_thickness_init = 0.1
Ti_func_init = lambda z: (z*Tis_init + (ice_thickness_init-z)*To(0.0))/ice_thickness_init
Ts_func_init = lambda z: (z*Tsa_init + (snow_thickness_init-z)*Tis_init)/snow_thickness_init

Ni = 20
Ns = 10
dzi_init = np.arange(Ni, 0, -1)/sum(np.arange(Ni, 0, -1))*ice_thickness_init
dzs_init = np.full(Ns, snow_thickness_init/Ns)
ice_cells_init = dzi_init.cumsum() - dzi_init/2
snow_cells_init = dzs_init.cumsum() - dzs_init/2
salinity_init = np.linspace(4.0, 1.0, Ni)

In [ ]:
dzi_arr, dzs_arr,\
timeline,\
temp_oi_arr, temp_ice_arr, temp_is_arr, temp_snow_arr, temp_sa_arr,\
rho_ice_arr,\
snow_filter = main_process\
(
    time_step=3600,
    time_end=3600*250,
    N_pseudoiter=50,
    Ti_init=Ti_func_init(ice_cells_init),#To(0.0) + np.arange(0.5, Ni)/Ni * (Tis_init - To(0.0)),
    Ts_init=Ts_func_init(snow_cells_init),#Tis_init + np.arange(0.5, Ns)/Ns * (Tsa_init - Tis_init),
    Toi_init=To(0.0),
    Tis_init=Tis_init,
    Tsa_init=Tsa_init,
    dzi_init=dzi_init,
    dzs_init=dzs_init,
    salinity=salinity_init,
    snow_thickness_threshold=0.05               
)

In [ ]:
vis_1 = uv.animate(dzi_arr, dzs_arr,
                   temp_oi_arr, temp_ice_arr, temp_is_arr, temp_snow_arr, temp_sa_arr,
                   timeline,
                   rho_ice_arr, rho_w, rho_s,
                   snow_filter)

In [ ]:
vis_1.save('imgs/freeze_without_prec.mp4', dpi=150)

### 2. Аналогично (1), но с осадками

In [ ]:
To = lambda t: Tf_w
Ta = lambda t: -30.0
p = lambda t: 1e-7
F_sw = lambda t: 100.0
u_a = lambda t: 15.0

Tis_init = -15.0
Tsa_init = -20.0
ice_thickness_init = 2.0
snow_thickness_init = 0.1
Ni = 20
Ns = 10
dzi_init = np.full(Ni, ice_thickness_init/Ni)
dzs_init = np.full(Ns, snow_thickness_init/Ns)
salinity_init = np.linspace(4.0, 1.0, Ni)

In [ ]:
dzi_arr, dzs_arr,\
timeline,\
temp_oi_arr, temp_ice_arr, temp_is_arr, temp_snow_arr, temp_sa_arr,\
rho_ice_arr,\
snow_filter = main_process\
(
    time_step=3600,
    time_end=3600*250,
    N_pseudoiter=50,
    Ti_init=To(0.0) + np.arange(0.5, Ni)/Ni * (Tis_init - To(0.0)),
    Ts_init=Tis_init + np.arange(0.5, Ns)/Ns * (Tsa_init - Tis_init),
    Toi_init=To(0.0),
    Tis_init=Tis_init,
    Tsa_init=Tsa_init,
    dzi_init=dzi_init,
    dzs_init=dzs_init,
    salinity=salinity_init,
    snow_thickness_threshold=0.05               
)

In [ ]:
vis_2 = uv.animate(dzi_arr, dzs_arr,
                   temp_oi_arr, temp_ice_arr, temp_is_arr, temp_snow_arr, temp_sa_arr,
                   timeline,
                   rho_ice_arr, rho_w, rho_s,
                   snow_filter)

In [ ]:
vis_2.save('imgs/freeze_with_prec.mp4', dpi=150)

### 3. Таяние льда без осадков

In [ ]:
melt_time_end=3600*240

To = lambda t: Tf_w
Ta = lambda t: (-20.0 + 40.0*t/melt_time_end) if t < melt_time_end else 20.0 
p = lambda t: 0.0
F_sw = lambda t: 100.0
u_a = lambda t: 15.0

Tis_init = -15.0
Tsa_init = -20.0
ice_thickness_init = 2.0
snow_thickness_init = 0.1
Ni = 20
Ns = 10
dzi_init = np.full(Ni, ice_thickness_init/Ni)
dzs_init = np.full(Ns, snow_thickness_init/Ns)
salinity_init = np.linspace(4.0, 1.0, Ni)

In [ ]:
X = np.linspace(0, 500, 50)

plt.figure(figsize=(15, 10))
plt.title('Temperature evolution', size=25)
plt.plot(X, np.array([Ta(t*3600) for t in X]))
plt.xlabel('time', size=20)
plt.ylabel(r'$T, ^\circ C$', size=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.grid()
plt.savefig('imgs/temperature_evo_3.png')
plt.show()

In [ ]:
dzi_arr, dzs_arr,\
timeline,\
temp_oi_arr, temp_ice_arr, temp_is_arr, temp_snow_arr, temp_sa_arr,\
rho_ice_arr,\
snow_filter = main_process\
(
    time_step=3600,
    time_end=3600*400,
    N_pseudoiter=50,
    Ti_init=To(0.0) + np.arange(0.5, Ni)/Ni * (Tis_init - To(0.0)),
    Ts_init=Tis_init + np.arange(0.5, Ns)/Ns * (Tsa_init - Tis_init),
    Toi_init=To(0.0),
    Tis_init=Tis_init,
    Tsa_init=Tsa_init,
    dzi_init=dzi_init,
    dzs_init=dzs_init,
    salinity=salinity_init,
    snow_thickness_threshold=0.05
)

In [ ]:
vis_3 = uv.animate(dzi_arr, dzs_arr,
                   temp_oi_arr, temp_ice_arr, temp_is_arr, temp_snow_arr, temp_sa_arr,
                   timeline,
                   rho_ice_arr, rho_w, rho_s,
                   snow_filter)

In [ ]:
vis_3.save('imgs/melting_without_prec.mp4', dpi=150)

### 4. Аналогично (3), но с осадками

In [ ]:
melt_time_end=3600*240

To = lambda t: Tf_w
Ta = lambda t: (-20.0 + 40.0*t/melt_time_end) if t < melt_time_end else 20.0 
p = lambda t: 1e-7
F_sw = lambda t: 100.0
u_a = lambda t: 15.0

Tis_init = -15.0
Tsa_init = -20.0
ice_thickness_init = 2.0
snow_thickness_init = 0.1
Ni = 20
Ns = 10
dzi_init = np.full(Ni, ice_thickness_init/Ni)
dzs_init = np.full(Ns, snow_thickness_init/Ns)
salinity_init = np.linspace(4.0, 1.0, Ni)

In [ ]:
dzi_arr, dzs_arr,\
timeline,\
temp_oi_arr, temp_ice_arr, temp_is_arr, temp_snow_arr, temp_sa_arr,\
rho_ice_arr,\
snow_filter = main_process\
(
    time_step=3600,
    time_end=3600*400,
    N_pseudoiter=50,
    Ti_init=To(0.0) + np.arange(0.5, Ni)/Ni * (Tis_init - To(0.0)),
    Ts_init=Tis_init + np.arange(0.5, Ns)/Ns * (Tsa_init - Tis_init),
    Toi_init=To(0.0),
    Tis_init=Tis_init,
    Tsa_init=Tsa_init,
    dzi_init=dzi_init,
    dzs_init=dzs_init,
    salinity = salinity_init,
    snow_thickness_threshold=0.05
)

In [ ]:
vis_4 = uv.animate(dzi_arr, dzs_arr,
                   temp_oi_arr, temp_ice_arr, temp_is_arr, temp_snow_arr, temp_sa_arr,
                   timeline,
                   rho_ice_arr, rho_w, rho_s,
                   snow_filter)

In [ ]:
vis_4.save('img\melting_with_prec.mp4', dpi=150)

### 5. Постепенное нарастание снежного покрова

In [ ]:
freeze_time_end=3600*240

To = lambda t: Tf_w
Ta = lambda t: (10.0 - 30.0*t/freeze_time_end) if t < freeze_time_end else -20.0 
p = lambda t: 1e-7
F_sw = lambda t: 100.0
u_a = lambda t: 15.0

Tis_init = -10.0
Tsa_init = -10.0
ice_thickness_init = 2.0
snow_thickness_init = 0.0
Ni = 20
Ns = 10
dzi_init = np.full(Ni, ice_thickness_init/Ni)
dzs_init = np.full(Ns, snow_thickness_init/Ns)

In [ ]:
X = np.linspace(0, 500, 50)

plt.figure(figsize=(15, 10))
plt.plot(X, np.array([Ta(t*3600) for t in X]))
plt.xlabel('time')
plt.ylabel(r'$T, ^\circ C$')
plt.grid()
plt.show()

In [ ]:
dzi_arr, dzs_arr,\
timeline,\
temp_oi_arr, temp_ice_arr, temp_is_arr, temp_snow_arr, temp_sa_arr,\
rho_ice_arr,\
snow_filter = main_process\
(
    time_step=3600,
    time_end=3600*500,
    N_pseudoiter=50,
    Ti_init=To(0.0) + np.arange(0.5, Ni)/Ni * (Tis_init - To(0.0)),
    Ts_init=Tis_init + np.arange(0.5, Ns)/Ns * (Tsa_init - Tis_init),
    Toi_init=To(0.0),
    Tis_init=Tis_init,
    Tsa_init=Tsa_init,
    dzi_init=dzi_init,
    dzs_init=dzs_init,
    snow_thickness_threshold=0.05               
)

In [ ]:
vis_5 = uv.animate(dzi_arr, dzs_arr,
                   temp_oi_arr, temp_ice_arr, temp_is_arr, temp_snow_arr, temp_sa_arr,
                   timeline,
                   rho_ice_arr, rho_w, rho_s,
                   snow_filter)

vis_5

## 6. Круглогодичный эксперимент

In [ ]:
def linear_year_exp(winter_time, spring_time, summer_time, autumn_time,
                    lowest_temp, highest_temp,
                    lowest_p, highest_p,
                    t):
    t %= (winter_time + spring_time + summer_time + autumn_time)
    if t < winter_time: # winter
        return lowest_temp, highest_p
    elif t < (winter_time + spring_time): # spring
        return lowest_temp + (t - winter_time)*(highest_temp - lowest_temp)/spring_time,\
               highest_p + (t - winter_time)*(lowest_p - highest_p)/spring_time
    elif t < (winter_time + spring_time + summer_time): # summer
        return highest_temp, lowest_p
    else: # autumn
        return highest_temp + (t - (winter_time + spring_time + summer_time))\
                             *(lowest_temp - highest_temp)/autumn_time,\
               lowest_p + (t - (winter_time + spring_time + summer_time))*(highest_p - lowest_p)/autumn_time

In [ ]:
winter_time = 350*3600.0
spring_time = 200*3600.0
summer_time = 150*3600.0
autumn_time = 200*3600.0
year_time = winter_time + spring_time + summer_time + autumn_time

lowest_temp = -30.0
highest_temp = 10.0
lowest_p = 0.0
highest_p = 1e-8

To = lambda t: Tf_w
Ta = lambda t: linear_year_exp(winter_time, spring_time, summer_time, autumn_time,
                               lowest_temp, highest_temp,
                               lowest_p, highest_p, t)[0]
p = lambda t: linear_year_exp(winter_time, spring_time, summer_time, autumn_time,
                               lowest_temp, highest_temp,
                               lowest_p, highest_p, t)[1]
F_sw = lambda t: 100.0
u_a = lambda t: 15.0

Tis_init = -15.0
Tsa_init = -20.0
ice_thickness_init = 4.0
snow_thickness_init = 0.1
Ni = 30
Ns = 5
dzi_init = np.full(Ni, ice_thickness_init/Ni)
dzs_init = np.full(Ns, snow_thickness_init/Ns)
salinity_init = np.linspace(4.0, 1.0, Ni)

In [ ]:
time_arr = np.linspace(0, 3*year_time, 100)

fig, ax1 = plt.subplots(figsize=(15, 10))
ax1.set_title('Temperature and precipitation evolution', size=25)
ax2 = ax1.twinx()
ax1.plot(time_arr/3600.0,
         [linear_year_exp(winter_time, spring_time, summer_time, autumn_time,
                          lowest_temp, highest_temp,
                          lowest_p, highest_p, t)[0] for t in time_arr],
         label='tempreature', color='b')
ax2.plot(time_arr/3600.0,
         [linear_year_exp(winter_time, spring_time, summer_time, autumn_time,
                          lowest_temp, highest_temp,
                          lowest_p, highest_p, t)[1] for t in time_arr],
         label='precipitation', color='r')
ax1.spines['bottom'].set_color('b')
ax1.set_xlabel('Time, h', size=15)
ax1.set_ylabel(r'Temperature, $^{\circ}C$', size=15)
ax2.set_ylabel(r'Precipitation, m/s', size=15)
ax1.tick_params(axis='both', labelsize=15)
ax2.tick_params(axis='y', labelsize=15)
ax2.spines['left'].set_color('b')
ax1.tick_params(axis='y', colors='b')
ax1.yaxis.label.set_color('b')
ax2.spines['right'].set_color('red')
ax2.tick_params(axis='y', colors='red')
ax2.yaxis.label.set_color('red')
ax1.grid()
plt.savefig('imgs/temp_and_prec.png')
plt.show()

In [ ]:
dzi_arr, dzs_arr,\
timeline,\
temp_oi_arr, temp_ice_arr, temp_is_arr, temp_snow_arr, temp_sa_arr,\
rho_ice_arr,\
snow_filter = main_process\
(
    time_step=3600.0,
    time_end=3*year_time,
    N_pseudoiter=25,
    Ti_init=To(0.0) + np.arange(0.5, Ni)/Ni * (Tis_init - To(0.0)),
    Ts_init=Tis_init + np.arange(0.5, Ns)/Ns * (Tsa_init - Tis_init),
    Toi_init=To(0.0),
    Tis_init=Tis_init,
    Tsa_init=Tsa_init,
    dzi_init=dzi_init,
    dzs_init=dzs_init,
    salinity=salinity_init,
    snow_thickness_threshold=0.05
)

In [ ]:
end_proc = -1

vis_year = uv.animate(dzi_arr[:end_proc], dzs_arr[:end_proc],
                      temp_oi_arr[:end_proc], temp_ice_arr[:end_proc], temp_is_arr[:end_proc], temp_snow_arr[:end_proc], temp_sa_arr[:end_proc],
                      timeline[:end_proc],
                      rho_ice_arr[:end_proc], rho_w, rho_s,
                      snow_filter[:end_proc])

vis_year

In [ ]:
vis_year.save("3year.mp4", dpi=200)

In [ ]:
def sec_order(arr, dz):
    first_order = np.gradient(arr, np.cumsum(dz) - dz/2, edge_order=2)
    second_order = np.gradient(first_order, np.cumsum(dz) - dz/2, edge_order=2)
    return second_order

In [ ]:
avg_abs_secord_winter = np.average([np.abs(sec_order(temp_ice, dzi))[1:-1]\
                                    for temp_ice, dzi\
                                    in zip(np.concatenate((temp_ice_arr[:350],
                                                          temp_ice_arr[900:1250],
                                                          temp_ice_arr[1800:2150])),
                                           np.concatenate((dzi_arr[:350],
                                                          dzi_arr[900:1250],
                                                          dzi_arr[1800:2150])))],
                                   axis=0)

avg_abs_secord_spring = np.average([np.abs(sec_order(temp_ice, dzi))[1:-1]\
                                    for temp_ice, dzi\
                                    in zip(np.concatenate((temp_ice_arr[350:550],
                                                          temp_ice_arr[1250:1450],
                                                          temp_ice_arr[2150:2350])),
                                           np.concatenate((dzi_arr[350:550],
                                                          dzi_arr[1250:1450],
                                                          dzi_arr[2150:2350])))],
                                   axis=0)

avg_abs_secord_summer = np.average([np.abs(sec_order(temp_ice, dzi))[1:-1]\
                                    for temp_ice, dzi\
                                    in zip(np.concatenate((temp_ice_arr[550:700],
                                                          temp_ice_arr[1450:1600],
                                                          temp_ice_arr[2350:2500])),
                                           np.concatenate((dzi_arr[550:700],
                                                          dzi_arr[1450:1600],
                                                          dzi_arr[2350:2500])))],
                                   axis=0)

avg_abs_secord_autumn = np.average([np.abs(sec_order(temp_ice, dzi))[1:-1]\
                                    for temp_ice, dzi\
                                    in zip(np.concatenate((temp_ice_arr[700:900],
                                                          temp_ice_arr[1600:1800],
                                                          temp_ice_arr[2500:2700])),
                                           np.concatenate((dzi_arr[700:900],
                                                          dzi_arr[1600:1800],
                                                          dzi_arr[2500:2700])))],
                                   axis=0)

avg_abs_secord_year = np.average([np.abs(sec_order(temp_ice, dzi))[1:-1]\
                                    for temp_ice, dzi\
                                    in zip(temp_ice_arr, dzi_arr)],
                                   axis=0)

In [ ]:
plt.figure(figsize=(15, 10))
plt.title("Average magnitude of temperature second derivative", size=25)
plt.plot(avg_abs_secord_winter, np.linspace(1/Ni, 1-1/Ni, Ni-2), label='winter')
plt.plot(avg_abs_secord_spring, np.linspace(1/Ni, 1-1/Ni, Ni-2), label='spring')
plt.plot(avg_abs_secord_summer, np.linspace(1/Ni, 1-1/Ni, Ni-2), label='summer')
plt.plot(avg_abs_secord_autumn, np.linspace(1/Ni, 1-1/Ni, Ni-2), label='autumn')
plt.plot(avg_abs_secord_year, np.linspace(1/Ni, 1-1/Ni, Ni-2), label='year', lw=5, alpha=0.5)
plt.xlabel(r'Second derivative of temperature, $^{\circ}C/m^2$', size=20)
plt.ylabel('Sigma-coordinate', size=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.legend(prop={'size':20})
plt.grid()
plt.savefig('imgs/secord_magnitude.png')
plt.show()

## 7. Точное решение для круглогодичного эксперимента

In [ ]:
Tis_init = -15.0
Tsa_init = -20.0
ice_thickness_init = 4.0
snow_thickness_init = 0.1
Ni = 100
Ns = 10
Ti_func_init = lambda z: (z*Tis_init + (ice_thickness_init-z)*To(0.0))/ice_thickness_init
Ts_func_init = lambda z: (z*Tsa_init + (snow_thickness_init-z)*Tis_init)/snow_thickness_init
ice_cells_init = dzi_init.cumsum() - dzi_init/2
snow_cells_init = dzs_init.cumsum() - dzs_init/2
salinity_init = np.linspace(4.0, 1.0, Ni)

dzi_arr_exact, dzs_arr_exact,\
timeline_exact,\
temp_oi_arr_exact, temp_ice_arr_exact, temp_is_arr_exact, temp_snow_arr_exact, temp_sa_arr_exact,\
rho_ice_arr_exact,\
snow_filter_exact = main_process\
(
    time_step=3600.0,
    time_end=3*year_time,
    N_pseudoiter=25,
    Ti_init=Ti_func_init(ice_cells_init),
    Ts_init=Ts_func_init(snow_cells_init),
    Toi_init=To(0.0),
    Tis_init=Tis_init,
    Tsa_init=Tsa_init,
    dzi_init=dzi_init,
    dzs_init=dzs_init,
    salinity=salinity_init,
    snow_thickness_threshold=0.05
)

In [ ]:
end_proc = -1

vis_year_exact = uv.animate(dzi_arr_exact[:end_proc], dzs_arr_exact[:end_proc],
                      temp_oi_arr_exact[:end_proc], temp_ice_arr_exact[:end_proc], temp_is_arr_exact[:end_proc], temp_snow_arr_exact[:end_proc], temp_sa_arr_exact[:end_proc],
                      timeline_exact[:end_proc],
                      rho_ice_arr_exact[:end_proc], rho_w, rho_s,
                      snow_filter_exact[:end_proc])

In [ ]:
%%time
vis_year_exact.save('year_exact.mp4', dpi=150)

In [ ]:
def thickening(N, s):
    if s == 0:
        return np.full(N, 1/N)
    else:
        return np.array([s*(1 - s)**i/(1 - (1 - s)**N) for i in range(N)])

In [ ]:
N = 15

plt.figure(figsize=(10, 6))
for s in np.arange(0, 0.16, 0.025):
    plt.plot([s]*(N+1), np.insert(np.cumsum(thickening(N, s)), 0,  0.0), marker='x', color='black')
plt.xticks(np.arange(0, 0.16, 0.025))
plt.xlabel('Level of thickening', size=20)
plt.ylabel('Sigma-coordinate', size=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.grid()
plt.savefig('imgs/level_of_thickening.png')
plt.grid(ls='--')
plt.show()

In [ ]:
Ni = 8

dzi_arr_map = {}
dzs_arr_map = {}
timeline_map = {}
temp_oi_arr_map = {}
temp_ice_arr_map = {}
temp_is_arr_map = {}
temp_snow_arr_map = {}
temp_sa_arr_map = {}
rho_ice_arr_map = {}
snow_filter_map = {}

for s in tqdm(np.arange(0, 0.21, 0.05)):
    dzi_init = thickening(Ni, s)*ice_thickness_init
    ice_cells_init = dzi_init.cumsum() - dzi_init/2
    dzi_arr, dzs_arr,\
    timeline,\
    temp_oi_arr, temp_ice_arr, temp_is_arr, temp_snow_arr, temp_sa_arr,\
    rho_ice_arr,\
    snow_filter = main_process\
    (
        time_step=3600.0,
        time_end=3*year_time,
        N_pseudoiter=25,
        Ti_init=Ti_func_init(ice_cells_init),
        Ts_init=Tis_init + np.arange(0.5, Ns)/Ns * (Tsa_init - Tis_init),
        Toi_init=To(0.0),
        Tis_init=Tis_init,
        Tsa_init=Tsa_init,
        dzi_init=dzi_init,
        dzs_init=dzs_init,
        salinity=salinity_init,
        snow_thickness_threshold=0.05
    )
    
    dzi_arr_map[s] = dzi_arr
    dzs_arr_map[s] = dzs_arr
    timeline_map[s] = timeline
    temp_oi_arr_map[s] = temp_oi_arr
    temp_ice_arr_map[s] = temp_ice_arr
    temp_is_arr_map[s] = temp_is_arr
    temp_snow_arr_map[s] = temp_snow_arr
    temp_sa_arr_map[s] = temp_sa_arr
    rho_ice_arr_map[s] = rho_ice_arr
    snow_filter_map[s] = snow_filter

In [ ]:
def h_error(exact_dz, test_dz):
    return sum(exact_dz) - sum(test_dz)

In [ ]:
def bottomline_error(exact_sol_dzi, exact_sol_dzs, exact_sol_rho_i,
                     test_sol_dzi, test_sol_dzs, test_sol_rho_i,
                     rho_s, rho_w):
    return bottomline(exact_sol_dzi, exact_sol_dzs, exact_sol_rho_i, rho_s, rho_w) - \
           bottomline(test_sol_dzi, test_sol_dzs, test_sol_rho_i, rho_s, rho_w)

In [ ]:
def T_error(exact_sol_dz, exact_sol_T, test_sol_dz, test_sol_T):
    test_sol_dz_new = exact_sol_dz*test_sol_dz.sum()/exact_sol_dz.sum()
    test_sol_T_new = np.interp(test_sol_dz_new.cumsum() - test_sol_dz_new/2,
                               test_sol_dz.cumsum() - test_sol_dz/2,
                               test_sol_T)
    return np.sqrt(np.mean(exact_sol_T - test_sol_T_new)**2)

In [ ]:
def bottomline(dzi, dzs, rho_i, rho_s, rho_w):
    return (-np.dot(dzi, rho_i) - rho_s*sum(dzs))/rho_w

In [ ]:
def twoD_error(exact_sol_dzi, exact_sol_dzs, exact_sol_T, exact_sol_rho_i,
               test_sol_dzi, test_sol_dzs, test_sol_T, test_sol_rho_i,
               rho_s, rho_w):
    test_sol_dzi_new = exact_sol_dzi*test_sol_dzi.sum()/exact_sol_dzi.sum()
    test_sol_T_new = np.interp(test_sol_dzi_new.cumsum() - test_sol_dzi_new/2,
                               test_sol_dzi.cumsum() - test_sol_dzi/2,
                               test_sol_T)
    exact_bottomline = bottomline(exact_sol_dzi, exact_sol_dzs, exact_sol_rho_i, rho_s, rho_w)
    test_bottomline = bottomline(test_sol_dzi, test_sol_dzs, test_sol_rho_i, rho_s, rho_w)
    exact_Z = np.cumsum(exact_sol_dzi) - exact_sol_dzi/2 + exact_bottomline
    test_Z = np.cumsum(test_sol_dzi_new) - test_sol_dzi_new/2 + test_bottomline
    return np.sqrt(np.mean([((e_Z - t_Z)/(exact_Z.max() - exact_Z.min()))**2 + \
                            ((e_T - t_T)/(exact_sol_T.max() - exact_sol_T.min()))**2 \
                            for e_Z, e_T, t_Z, t_T \
                            in zip(exact_Z, exact_sol_T, test_Z, test_sol_T_new)]))

In [ ]:
plt.figure(figsize=(40, 10))
plt.title('Thickness error', size=25)
for s in tqdm(dzi_arr_map.keys()):
    plt.plot(np.array(timeline_map[s])/3600.0,
             [h_error(dzi_exact, dzi_test) for dzi_exact, dzi_test in zip(dzi_arr_exact, dzi_arr_map[s])],
             label="s = {:.2f}".format(s))
plt.xlabel('time, h.', size=20)
plt.ylabel(r'$Error, m.$', size=20)
plt.xticks(np.arange(0, timeline_map[s][-1]/3600.0, 100), fontsize=15)
plt.yticks(fontsize=15)
plt.legend(prop={'size':20})
plt.grid()
plt.savefig('imgs/errors_h.png')
plt.plot()

In [ ]:
plt.figure(figsize=(40, 10))
plt.title('T error', size=25)
for s in tqdm(dzi_arr_map.keys()):
    plt.plot(np.array(timeline_map[s])/3600.0,
             [T_error(dzi_exact, T_exact, dzi_test, T_test)\
              for dzi_exact, T_exact, dzi_test, T_test\
              in zip(dzi_arr_exact, temp_ice_arr_exact, dzi_arr_map[s], temp_ice_arr_map[s])],
             label="s = {:.2f}".format(s))
plt.xlabel('time, h.', size=20)
plt.ylabel(r'$Error, ^\circ C$', size=20)
plt.xticks(np.arange(0, timeline_map[s][-1]/3600.0, 100), fontsize=15)
plt.yticks(fontsize=15)
plt.legend(prop={'size':20})
plt.grid()
plt.savefig('imgs/errors_T.png')
plt.plot()

In [ ]:
plt.figure(figsize=(15, 10))
plt.plot(dzi_arr_exact[-1].cumsum() - dzi_arr_exact[-1]/2, temp_ice_arr_exact[-1], label='exact sol')
plt.plot(dzi_arr_map[0.2][-1].cumsum() - dzi_arr_map[0.2][-1]/2, temp_ice_arr_map[0.2][-1], label='test sol')
plt.legend()
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(40, 10))
plt.title('2D error')
for s in tqdm(np.arange(0, 0.16, 0.025)):
    plt.plot(np.array(timeline_map[s])/3600.0,
             [twoD_error(dzi_exact, dzs_exact, T_exact, rho_i_exact,
                         dzi_test, dzs_test, T_test, rho_i_test,
                         rho_s, rho_w)\
              for dzi_exact, dzs_exact, T_exact, rho_i_exact, dzi_test, dzs_test, T_test, rho_i_test\
              in zip(dzi_arr_exact, dzs_arr_exact, temp_ice_arr_exact, rho_ice_arr_exact,
                     dzi_arr_map[s], dzs_arr_map[s], temp_ice_arr_map[s], rho_ice_arr_map[s])],
             label="s = {:.2f}".format(s))
plt.legend()
plt.grid()
plt.plot()

In [ ]:
plt.figure(figsize=(40, 10))
plt.title('Bottomline error')
for s in tqdm(dzi_arr_map.keys()):
    plt.plot(np.array(timeline_map[s])/3600.0, 
             [bottomline_error(dzi_exact, dzs_exact, rho_i_exact,
                               dzi_test, dzs_test, rho_i_test,
                               rho_s, rho_w)\
              for dzi_exact, dzs_exact, rho_i_exact, dzi_test, dzs_test, rho_i_test\
              in zip(dzi_arr_exact, dzs_arr_exact, rho_ice_arr_exact,
                     dzi_arr_map[s], dzs_arr_map[s], rho_ice_arr_map[s])],
             label="s = {:.2f}".format(s))
plt.legend()
plt.grid()
plt.savefig('imgs/errors_bottomline.png')
plt.plot()

In [ ]:
end_proc = -1
s = list(dzi_arr_map.keys())[-2]

vis_year_test = uv.animate(dzi_arr_map[s][:end_proc], dzs_arr_map[s][:end_proc],
                           temp_oi_arr_map[s][:end_proc], temp_ice_arr_map[s][:end_proc],
                           temp_is_arr_map[s][:end_proc], temp_snow_arr_map[s][:end_proc],
                           temp_sa_arr_map[s][:end_proc],
                           timeline_map[s][:end_proc],
                           rho_ice_arr_map[s][:end_proc], rho_w, rho_s,
                           snow_filter_map[s][:end_proc])

In [ ]:
vis_year_test.save('imgs/thick_3year.mp4', dpi=150)

In [ ]:
ice_cells_init

In [ ]:
s = 0.15

dzi_arr_map_n = {}
dzs_arr_map_n = {}
timeline_map_n = {}
temp_oi_arr_map_n = {}
temp_ice_arr_map_n = {}
temp_is_arr_map_n = {}
temp_snow_arr_map_n = {}
temp_sa_arr_map_n = {}
rho_ice_arr_map_n = {}
snow_filter_map_n = {}

for Ni in tqdm([5, 8, 10, 15, 20]):
    dzi_init = thickening(Ni, s)*ice_thickness_init
    ice_cells_init = dzi_init.cumsum() - dzi_init/2
    salinity_init = np.linspace(4.0, 1.0, Ni)
    dzi_arr, dzs_arr,\
    timeline,\
    temp_oi_arr, temp_ice_arr, temp_is_arr, temp_snow_arr, temp_sa_arr,\
    rho_ice_arr,\
    snow_filter = main_process\
    (
        time_step=3600.0,
        time_end=3*year_time,
        N_pseudoiter=25,
        Ti_init=Ti_func_init(ice_cells_init),
        Ts_init=Tis_init + np.arange(0.5, Ns)/Ns * (Tsa_init - Tis_init),
        Toi_init=To(0.0),
        Tis_init=Tis_init,
        Tsa_init=Tsa_init,
        dzi_init=dzi_init,
        dzs_init=dzs_init,
        salinity=salinity_init,
        snow_thickness_threshold=0.05
    )
    
    dzi_arr_map_n[Ni] = dzi_arr
    dzs_arr_map_n[Ni] = dzs_arr
    timeline_map_n[Ni] = timeline
    temp_oi_arr_map_n[Ni] = temp_oi_arr
    temp_ice_arr_map_n[Ni] = temp_ice_arr
    temp_is_arr_map_n[Ni] = temp_is_arr
    temp_snow_arr_map_n[Ni] = temp_snow_arr
    temp_sa_arr_map_n[Ni] = temp_sa_arr
    rho_ice_arr_map_n[Ni] = rho_ice_arr
    snow_filter_map_n[Ni] = snow_filter

In [ ]:
plt.figure(figsize=(40, 10))
plt.title('Thickness error', size=25)
for Ni in dzi_arr_map_n.keys():
    plt.plot(np.array(timeline_map_n[Ni])/3600.0,
             [h_error(dzi_exact, dzi_test) for dzi_exact, dzi_test\
              in zip(dzi_arr_exact, dzi_arr_map_n[Ni])],
             label="N = {}".format(Ni))
plt.xlabel('time, h.', size=20)
plt.ylabel(r'$Error, m.$', size=20)
plt.xticks(np.arange(0, timeline_map_n[Ni][-1]/3600.0, 100), fontsize=15)
plt.yticks(fontsize=15)
plt.legend(prop={'size':20})
plt.grid()
plt.savefig('imgs/N_errors_h.png')
plt.plot()

In [ ]:
plt.figure(figsize=(40, 10))
plt.title('T error', size=25)
for Ni in dzi_arr_map_n.keys():
    plt.plot(np.array(timeline_map_n[Ni])/3600.0,
             [T_error(dzi_exact, T_exact, dzi_test, T_test)\
              for dzi_exact, T_exact, dzi_test, T_test\
              in zip(dzi_arr_exact, temp_ice_arr_exact, dzi_arr_map_n[Ni], temp_ice_arr_map_n[Ni])],
             label="N = {}".format(Ni))
plt.xlabel('time, h.', size=20)
plt.ylabel(r'$Error, ^\circ C$', size=20)
plt.xticks(np.arange(0, timeline_map_n[Ni][-1]/3600.0, 100), fontsize=15)
plt.yticks(fontsize=15)
plt.legend(prop={'size':20})
plt.grid()
plt.savefig('imgs/N_errors_T.png')
plt.plot()

In [ ]:
plt.figure(figsize=(40, 10))
plt.title('Bottomline error')
for Ni in tqdm(dzi_arr_map_n.keys()):
    plt.plot(np.array(timeline_map_n[Ni])/3600.0, 
             [bottomline_error(dzi_exact, dzs_exact, rho_i_exact,
                               dzi_test, dzs_test, rho_i_test,
                               rho_s, rho_w)\
              for dzi_exact, dzs_exact, rho_i_exact, dzi_test, dzs_test, rho_i_test\
              in zip(dzi_arr_exact, dzs_arr_exact, rho_ice_arr_exact,
                     dzi_arr_map_n[Ni], dzs_arr_map_n[Ni], rho_ice_arr_map_n[Ni])],
             label="N = {}".format(Ni))
plt.legend()
plt.grid()
plt.savefig('imgs/N_errors_bottomline.png')
plt.plot()